Import Libraries

In [52]:
from pyvis.network import Network
import networkx as nx
import random
from networkx.algorithms.community import greedy_modularity_communities
from networkx.linalg.modularitymatrix import modularity_matrix, directed_modularity_matrix
import numpy as np
from scipy.linalg import eig

Create Helper Functions

In [53]:
# recursive function that will traverse the nodes
def createPaper(network, authors, probStop):
    '''
    Will take network, list of authors, and probStop as input
    '''
    currAuthorID = authors[-1]
    newNeighbors = set(network.neighbors(currAuthorID)).difference(set(authors))

    # base condition: stop at node if probStop hit or there are no new neighbors to traverse
    if random.random() < probStop or len(newNeighbors) == 0:
        return
    
    # create list reprsenting probabilities for the neighboring nodes of the current coauthor
    probs = []
    for neighbor in newNeighbors:
        nData = network.get_edge_data(currAuthorID, neighbor)
        probs.extend([neighbor] * nData["weight"])

    # Select coauthor from neighbors probabilities list
    coauthorID = random.choice(probs)

    # update all edges of coauthors to this new author
    for author in authors:
        # if there is not an edge, create one
        if not network.has_edge(author, coauthorID) and author != coauthorID:
            network.add_edge(author, coauthorID, weight=0, width=1)
        newWeight = network.get_edge_data(author, coauthorID)["weight"] + 1
        #network.update(edges=[ (author, coauthorID, {"weight": newWeight, "width": newWeight//2}) ])
        network.update(edges=[ (author, coauthorID, {"weight": newWeight}) ])

    # call function recursively with coauthor
    authors.append(coauthorID)
    createPaper(network, authors, probStop)


Define initial parameters

In [54]:
# define time steps
timeSteps = 15

# Probabilities
# probability that you generate new author
probNewAuthor = 0.5
# probability that you stop at a given node
probStop = 0.7

# define fields
fieldColors = {"CS": "blue", 
                "Math": "green", 
                "Physics": "red"}
fields = list(fieldColors.keys())

# define initial scholars, will be in form (id, scholarField, color)
scholarField = random.choice(fields)
nodeID = 0


Create Model

In [55]:
network = nx.Graph()
scholarField = random.choice(fields)
network.add_node(nodeID, label=scholarField, color=fieldColors[scholarField])
# go through time steps, add new scholar and paper at each step
for i in range(1, timeSteps):

    # Choose first author, either new scholar or random choice
    currNodes = list(network.nodes())
    authors = [random.choice(currNodes)]

    # with probability, add new author to network set as main author with a coauthor
    if random.random() < probNewAuthor:
        # generate author and field
        scholarField = random.choice(fields)
        nodeID += 1
        author = nodeID
        network.add_node(author, label=scholarField, color=fieldColors[scholarField])

        # generate random coauthor from currNodes, which doesn't have the new node added in
        coauthorID = random.choice(currNodes)
        network.add_edge(author, coauthorID, weight=1, width=1)

        # update authors list
        authors = [author, coauthorID]

    # Add new paper, calling function
    createPaper(network, authors, probStop)

Display Network

In [56]:
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 2, 'width': 1}), (0, 2, {'weight': 1, 'width': 1}), (0, 3, {'weight': 1, 'width': 1}), (0, 4, {'weight': 1, 'width': 1}), (1, 2, {'weight': 3, 'width': 1}), (1, 5, {'weight': 1, 'width': 1}), (2, 5, {'weight': 1, 'width': 1}), (3, 6, {'weight': 2, 'width': 1}), (5, 8, {'weight': 1, 'width': 1}), (6, 7, {'weight': 2, 'width': 1}), (6, 9, {'weight': 1, 'width': 1}), (7, 9, {'weight': 1, 'width': 1})]


### Show modularity communities

In [57]:
communities = greedy_modularity_communities(network)
print(communities)
# look at exactly whay the matrix is calculating, make sure it is the unweighted
matrix = modularity_matrix(network)
print(matrix)

[frozenset({8, 1, 2, 5}), frozenset({0, 3, 4}), frozenset({9, 6, 7})]
[[-0.66666667  0.5         0.5         0.66666667  0.83333333 -0.5
  -0.5        -0.33333333 -0.16666667 -0.33333333]
 [ 0.5        -0.375       0.625      -0.25       -0.125       0.625
  -0.375      -0.25       -0.125      -0.25      ]
 [ 0.5         0.625      -0.375      -0.25       -0.125       0.625
  -0.375      -0.25       -0.125      -0.25      ]
 [ 0.66666667 -0.25       -0.25       -0.16666667 -0.08333333 -0.25
   0.75       -0.16666667 -0.08333333 -0.16666667]
 [ 0.83333333 -0.125      -0.125      -0.08333333 -0.04166667 -0.125
  -0.125      -0.08333333 -0.04166667 -0.08333333]
 [-0.5         0.625       0.625      -0.25       -0.125      -0.375
  -0.375      -0.25        0.875      -0.25      ]
 [-0.5        -0.375      -0.375       0.75       -0.125      -0.375
  -0.375       0.75       -0.125       0.75      ]
 [-0.33333333 -0.25       -0.25       -0.16666667 -0.08333333 -0.25
   0.75       -0.16666667

In [58]:
# get eigenvalues and vectors of matrix, maybe use scipy instead
eVals, eVects = np.linalg.eig(matrix)
# find indices of largest eigenvalue, get corresponding eigenvector
# if no positive eigenvalue, then cannot divide network further because it will not improve modularity
if max(eVals) > 0:
    index = np.argmax(eVals)
    maxEvect = np.array(eVects)[index]
    # split eigenvector into positive and negative values, collecting vertices
    g1 = []
    g2 = []
    for j, val in enumerate(maxEvect):
        if val < 0:
            g1.append(j)
            # test update the color
            network.update(nodes=[(j, {"label": "g1", "color": "green"})])
        else:
            g2.append(j)
            network.update(nodes=[(j, {"label": "g2", "color": "red"})])
# print(g1)
# print(g2)
print(eVals)


[ 2.27834730e+00 -2.21902096e+00  1.20979677e+00  3.17915745e-01
 -2.66701504e-16  1.44621939e-01 -1.48166080e+00 -1.00000000e+00
 -1.00000000e+00 -1.00000000e+00]


In [59]:
# recreate network
nt = Network()
# populates the nodes and edges data structures
nt.from_nx(network)
print(network.edges.data())
nt.show('docs/models/modularity.html')

[(0, 1, {'weight': 2, 'width': 1}), (0, 2, {'weight': 1, 'width': 1}), (0, 3, {'weight': 1, 'width': 1}), (0, 4, {'weight': 1, 'width': 1}), (1, 2, {'weight': 3, 'width': 1}), (1, 5, {'weight': 1, 'width': 1}), (2, 5, {'weight': 1, 'width': 1}), (3, 6, {'weight': 2, 'width': 1}), (5, 8, {'weight': 1, 'width': 1}), (6, 7, {'weight': 2, 'width': 1}), (6, 9, {'weight': 1, 'width': 1}), (7, 9, {'weight': 1, 'width': 1})]
